In [0]:
# -*- coding: utf-8 -*-
 
 
from pyspark import SparkContext, SparkConf
from math import sqrt
from numpy.linalg import norm 
import numpy as np
import time
import sys
 
 
def computeDistance(x,y):
    return sqrt(sum([(a - b)**2 for a,b in zip(x,y)]))
 
 
def closestCluster(dist_list):
    cluster = dist_list[0][0]
    min_dist = dist_list[0][1]
    for elem in dist_list:
        if elem[1] < min_dist:
            cluster = elem[0]
            min_dist = elem[1]
    return (cluster,min_dist)
 
def sumList(x,y):
    return [x[i]+y[i] for i in range(len(x))]
 
def moyenneList(x,n):
    return [x[i]/n for i in range(len(x))]
 
 
def choice(p):
    
    #Generates a random sample from [0, len(p)),
    #where p[i] is the probability associated with i. 
    
    random = np.random.random()
    r = 0.0
    for idx in range(len(p)):
        r = r + p[idx]
        if r > random:
            return idx
    assert(False)
 
 
def KmeansOpti_init(rdd, K, RUNS):
    
    ########################################################
    # Select `RUNS` sets of initial points for `K`-means++ #
    ########################################################
    
    # the `centroides` variable is what we want to return
    n_data = rdd.count()
    #shape = rdd.take(1)[0][1].shape[0]
    centroides = np.zeros((RUNS, K, 4))
    
    def update_dist(vec, dist, k):
        new_dist = norm(vec - centroides[:, k], axis=1)**2
        return np.min([dist, new_dist], axis=0)
    
    # The second element `dist` in the tuple below is the
    # closest distance from each data point to the selected
    # points in the initial set
    data = (rdd.map(lambda p: (p, [np.inf] * RUNS)).cache())
    # Collect the feature vectors of all data points
    local_data = (rdd.map(lambda x: x[1]).collect())
    # Randomly select the first point for every run ofk-means++
    sample = [local_data[k] for k in
        np.random.randint(0, len(local_data), RUNS)]
    centroides[:, 0] = sample

    for idx in range(K - 1):
        #Update distance
        data = (data.map(lambda x: ((x[0][0],x[0][1]),update_dist(x[0][1],x[1],idx))).cache())
        #Calculate sum of D_i(x)^2
        d1 = data.map(lambda x: (1,x[1]))
        d2 = d1.reduceByKey(lambda x,y: np.sum([x,y], axis=0))
        total = d2.collect()[0][1]
        #Normalize each distance to get the probabilities and reshape
        prob = (data.map(lambda x: np.divide(x[1],total)).collect())
        prob = np.reshape(prob,(len(local_data), RUNS))
        #K'th centroid for each run
        data_id = [choice(prob[:,i]) for i in range(RUNS)]
        sample = [local_data[i] for i in data_id]
        centroides[:, idx+1] = sample
    return centroides
 
 
def KmeansOpti(data, nb_clusters, max_steps, RUNS, min_switch = 0, init="kmeans++"):
    clusteringDone = False
    number_of_steps = 0
    current_error = float("inf")
    # A broadcast value is sent to and saved  by each executor for further use
    # instead of being sent to each executor when needed.
    nb_elem = sc.broadcast(data.count())
 
    ##################################################
    # Select initial centroides with Kmeans++ method #
    ##################################################

    data_no_label = data.map(lambda x : (x[0],x[1][:4]))

    if init=="kmeans++":
        init=KmeansOpti_init(data_no_label,nb_clusters,RUNS)
        #create a unique key for each cluster of each run 
        #key = (Run_id,Cluster_id)
        centroides = sc.parallelize(init.reshape(RUNS*nb_clusters,4)).map(lambda x:[float(i) for i in x])\
              .zipWithIndex()\
              .map(lambda x: ((x[1]//nb_clusters,x[1]%nb_clusters),x[0]))
        print("Kmeans++ initialization done.")
        print("init centroides = ",centroides.collect())
    else:
        centroides = sc.parallelize(data_no_label.takeSample('withoutReplacment',nb_clusters*RUNS))\
              .zipWithIndex()\
              .map(lambda x: ((x[1]//nb_clusters,x[1]%nb_clusters),x[0][1][:-1]))
        print("Simple Kmeans initialization done.")
        print("init centroides = ",centroides.collect())

    #centroides example :
    #((0, 0), [4.9, 3.0, 1.4, 0.2])
    #((RUN, center), [4.9, 3.0, 1.4, 0.2])

    while not clusteringDone:
        
        start=time.time()
        print("step num = ",number_of_steps)
 
        #############################
        # Assign points to clusters #
        #############################
 
        joined = data.cartesian(centroides)
        # ((0, [5.1, 3.5, 1.4, 0.2, 'Iris-setosa']), ((0, 0), [4.4, 3.0, 1.3, 0.2]))
        # ((point, [5.1, 3.5, 1.4, 0.2, 'Iris-setosa']), ((RUN, center), [4.4, 3.0, 1.3, 0.2]))
 
        # We compute the distance between the points and each cluster
        dist = joined.map(lambda x: ((x[0][0],x[1][0][0]),(x[1][0][1], computeDistance(x[0][1][:-1], x[1][1]))))
        # ((0, 0), (0, 0.866025403784438))
        # ((point, RUN), (center, dist))
 
        dist_list = dist.groupByKey().mapValues(list)
        
        # ((0, 0), [(0, 0.866025403784438), (1, 3.7), (2, 0.5385164807134504)])
        # ((point, RUN), [(center1, dist1), (center2, dist2), (center3, dist3)])
 
        # We keep only the closest cluster to each point.
        min_dist = dist_list.mapValues(closestCluster).map(lambda x:((x[0][0], x[0][1]), (x[1][0], x[1][1])))
        # ((0, 0), (2, 0.5385164807134504))
 
        # assignment will be our return value : It contains the datapoint,
        # the id of the closest cluster and the distance of the point to the centroid
        assignment = min_dist.map(lambda x:(x[0][0], (x[0][1], x[1][0], x[1][1]))).join(data)
 
        # (0, ((0, 2, 0.5385164807134504), [5.1, 3.5, 1.4, 0.2, 'Iris-setosa']))
        # (point, ((RUN, center, dist), [5.1, 3.5, 1.4, 0.2, 'Iris-setosa']))
 
        ############################################
        # Compute the new centroid of each cluster #
        ############################################
 
        clusters = assignment.map(lambda x: ((x[1][0][0],x[1][0][1]), x[1][1][:-1]))
        # ((0, 2), [5.1, 3.5, 1.4, 0.2])
        # ((RUN, center), [5.1, 3.5, 1.4, 0.2])
        
        count = clusters.map(lambda x: (x[0],1)).reduceByKey(lambda x,y: x+y)
        somme = clusters.reduceByKey(sumList)
        centroidesCluster = somme.join(count).map(lambda x : (x[0],moyenneList(x[1][0],x[1][1])))
 
        ############################
        # Is the clustering over ? #
        ############################
 
        # Let's see how many points have switched clusters.
        if number_of_steps > 0:
            switchTAB=prev_assignment.join(min_dist).map(lambda x: (x[0][1],1) if (x[1][0][0] != x[1][1][0]) else (x[0][1],0)).reduceByKey(lambda x,y: x+y).map(lambda x: x[1])
            print(switchTAB.collect())
            switch=switchTAB.reduce(lambda x,y: x+y)
            print("total_switch = ",switch)
        else:
            switch = nb_elem.value*RUNS
        if switch <= min_switch or number_of_steps == max_steps:
            clusteringDone = True
            sum_dist = min_dist.map(lambda x: (x[0][1],x[1][1])).reduceByKey(lambda x,y: x + y)
            avg_dist = sum_dist.map(lambda x : x[1]).reduce(lambda x,y:x+y)/RUNS
            print("avg_dist = ",avg_dist)
            error=sum_dist.map(lambda x : sqrt(x[1])/nb_elem.value).collect()
            best_error=min(error)
            best_RUN=error.index(best_error)
            best_assignment=assignment.filter(lambda x:x[1][0][0]==best_RUN).map(lambda x:(x[0], ((x[1][0][1],x[1][0][2]), x[1][1])))
            avg_error=sum(error)/RUNS
            print(avg_error)
            worst_error=max(error)
            print(worst_error)
        else:
            centroides = centroidesCluster
            prev_assignment = min_dist
            number_of_steps += 1
        print("time = ",time.time() - start)
        print("avg_switch = ",switch)
 
    return (best_assignment, best_error, number_of_steps)
 
 
 
if __name__ == "__main__":
 
    conf = SparkConf().setAppName('exercice')
    sc = SparkContext(conf=conf)

    if len(sys.argv) != 6:
      print("You have to enter 5 arguments")
      print(" 1 Path to the data text file")
      print(" 2 Path to the output file")
      print(" 3 Number K of clusters")
      print(" 4 Number S of Steps")
      print(" 5 Number R of RUNS")
      print("for example : spark-submit KmeansOpti.py path/to/the/file/iris.data.txt 3 5 2")
      exit(0)

    # inputs
    input_path = sys.argv[1]  # path to the data file
    output_path = sys.argv[2]  # path to the output folder
    nb_clusters = int(sys.argv[3]) # number of clusters
    max_steps = int(sys.argv[4]) # maximum number of iterations
    RUNS = int(sys.argv[5]) # number of RUNS
    #min_switch = int(sys.argv[6]) # minimum number of switches needed to converge, default = 0
    #init = int(sys.argv[7]) # wich kind of initialization, default = "kmeans++""
 
    lines = sc.textFile(input_path)
    data = lines.map(lambda x: x.split(','))\
            .map(lambda x: [float(i) for i in x[:4]]+[x[4]])\
            .zipWithIndex()\
            .map(lambda x: (x[1],x[0]))
    # zipWithIndex allows us to give a specific index to each point
    # (0, [5.1, 3.5, 1.4, 0.2, 'Iris-setosa'])

    st = time.time()
    clusteringOpti = KmeansOpti(data,nb_clusters,max_steps,RUNS)

    execution_time = time.time() - st
    #unique identifier for our result file
    identifier = str(int(time.time())-1574248997)
    info = sc.parallelize(["id = " + identifier\
	    , "name = KmeansOpti.py"\
            , "RUNS = " + str(RUNS)\
            , "K clusters = " + str(nb_clusters)\
            , "max steps = " + str(max_steps)\
            ,"#######RESULT######################\n"\
            , "time = " + str(execution_time)\
            , "error = " + str(clusteringOpti[1])\
            , "steps = " + str(clusteringOpti[2])])

    print ("Time takes to converge with KmeansOpti:", execution_time )
    print("errorOpti=",clusteringOpti[1],"stepsOpti=",clusteringOpti[2])
    print("assignmentTOTAL=",clusteringOpti[0].count(),"assignment example",clusteringOpti[0].take(1))

    #save the result

    result_file = output_path+"/KmeansOpti"+identifier
    info_file = output_path+"/KmeansOpti"+identifier+"info"
    clusteringOpti[0].saveAsTextFile(result_file)
    # if you want to have only 1 file as a result
    result_file_unique = output_path+"/1KmeansOpti"+identifier
    clusteringOpti[0].coalesce(1).saveAsTextFile(result_file_unique)

    info.coalesce(1).saveAsTextFile(info_file)
 
    print(clusteringOpti)
